packages

In [1]:
import xarray as xr
import numpy as np 
import geopandas as gpd
import xagg as xa

data locations

In [2]:
loc_tc = "/Users/kevinschwarzwald/Downloads/xagg_mwe/TCs/"
loc_shp = "/Users/kevinschwarzwald/Downloads/xagg_mwe/State_Shapefile/"

read TC data

In [3]:
tc_winds = xr.open_dataarray(loc_tc+"TCs_example.nc")

Read shapefile

In [4]:
shp = gpd.read_file(loc_shp)

apply xa functions

In [7]:
weightmap = xa.pixel_overlaps(tc_winds,shp)

creating polygons for each pixel...
lat/lon bounds not found in dataset; they will be created.
HERE IS THE ISSUE PROBABLY
HERE IS THE ISSUE PROBABLY
calculating overlaps between pixels and output polygons...
success!


In [8]:
xa.aux.get_bnds(tc_winds.to_dataset(name='swath'))

lat/lon bounds not found in dataset; they will be created.
HERE IS THE ISSUE PROBABLY
HERE IS THE ISSUE PROBABLY


<xarray.Dataset>
Dimensions:   (lon: 532, lat: 348, nS: 10, bnds: 2)
Coordinates:
  * lon       (lon) float64 -115.0 -114.9 -114.8 -114.7 ... -62.1 -62.0 -61.9
  * lat       (lat) float64 19.95 20.05 20.15 20.25 ... 54.35 54.45 54.55 54.65
  * nS        (nS) int64 0 1 2 3 4 5 6 7 8 9
  * bnds      (bnds) int64 0 1
Data variables:
    swath     (nS, lat, lon) float64 ...
    lat_bnds  (lat, bnds) float64 19.9 20.0 20.0 20.1 ... 54.5 54.6 54.6 54.7
    lon_bnds  (lon, bnds) float64 -115.1 -115.0 -115.0 ... -61.95 -61.95 -61.85

In [11]:
ds = tc_winds.copy().to_dataset(name='swath')

edges={'lat':[-90,90],'lon':[-180,180]}
wrap_around_thresh=5

if ds.lon.max()>180:
    raise ValueError('Longitude seems to be in the 0:360 format.'+
                     ' -180:180 format required.')
    # Future versions should be able to work with 0:360 as well...
    # honestly, it *may* already work by just changing edges['lon']
    # to [0,360], but it's not tested yet. 

if ('lat' not in ds.keys()) | ('lon' not in ds.keys()):
    raise KeyError('"lat"/"lon" not found in [ds]. Make sure the '+
                   'geographic dimensions follow this naming convention.')


if 'lat_bnds' in ds.keys():
     print('bounds found')
else:
    print('lat/lon bounds not found in dataset; they will be created.')
    # Build lat / lon bound 
    for var in ['lat','lon']:
        bnds_tmp = xr.DataArray(data=np.zeros((ds.dims[var],2))*np.nan,
                                dims=[var,'bnds'],
                                coords=[ds[var],np.arange(0,2)])

        # Assign all non-edge bounds as just half of the distance from the center
        # of each pixel to the center of the next pixel
        print('HERE IS THE ISSUE PROBABLY')
        bnds_tmp[1:,:] = xr.concat([ds[var]-0.5*ds[var].diff(var),
                                      ds[var]+0.5*ds[var].diff(var)],dim='bnd').transpose(var,'bnd')

        # Fill in last missing band before edge cases (the inner band of the 
        # first pixel, which is just equal to the next edge)
        bnds_tmp[0,1] = bnds_tmp[1,0]
        #print(bnds_tmp)
        # Now deal with edge cases; basically either use the diff from the last
        # interval between pixels, or max out at 90. 
        if ds[var].diff(var)[0]>0:
            bnds_tmp[0,0] = np.max([edges[var][0],ds[var][0].values-0.5*(ds[var][1]-ds[var][0]).values])
            bnds_tmp[-1,1] = np.min([edges[var][1],ds[var][-1].values+0.5*(ds[var][-1]-ds[var][-2]).values])
        else:
            bnds_tmp[0,0] = np.min([edges[var][1],ds[var][0].values+0.5*(ds[var][1]-ds[var][0]).values])
            bnds_tmp[-1,1] = np.max([edges[var][0],ds[var][-1].values-0.5*(ds[var][-1]-ds[var][-2]).values])
        #print(bnds_tmp)

        # Fix crossing-over-360 issues in the lon
        if var == 'lon':
            # To be robust to partial grids; setting the rolled over edges equal to
            # each other if one of the edges is the -180/180 and the other one isn't, 
            # but 'close enough' (within 5 degrees + warning)
            if (bnds_tmp[0,0] in edges[var]) & (bnds_tmp[-1,1] not in edges[var]):
                # Make sure that the other edge is within the wrap-around threshold
                # (to avoid wrapping around if a grid is only -180:45 for example)
                if np.min(np.abs(bnds_tmp[-1,1].values-edges[var])) <= wrap_around_thresh:
                    if np.min(np.abs(bnds_tmp[-1,1].values-edges[var])) > ds[var].diff(var).max():
                        warnings.warn('Wrapping around '+[var]+' value of '+bnds_tmp[-1,1].values+', '+
                                      'because it is closer to a coordinate edge ('+
                                      ', '.join([str(n) for n in edges[var]])+') than the '+
                                      '[wrap_around_thresh] ('+str(wrap_around_thresh)+'); '+
                                      'however, it is farther away from that edge than the '+
                                      'maximum pixel width in the '+var+' direction. If this is '+
                                      'intended, no further action is necessary. Otherwise, reduce '+
                                      'the [wrap_around_thresh].')
                    bnds_tmp[0,0] = bnds_tmp[-1,1]
            elif (bnds_tmp[0,0] not in edges[var]) & (bnds_tmp[-1,1] in edges[var]):
                if np.min(np.abs(bnds_tmp[0,0].values-edges[var])) <= wrap_around_thresh:
                    if np.min(np.abs(bnds_tmp[0,0].values-edges[var])) > ds[var].diff(var).max():
                        warnings.warn('Wrapping around '+[var]+' value of '+bnds_tmp[0,0].values+', '+
                                      'because it is closer to a coordinate edge ('+
                                      ', '.join([str(n) for n in edges[var]])+') than the '+
                                      '[wrap_around_thresh] ('+str(wrap_around_thresh)+'); '+
                                      'however, it is farther away from that edge than the '+
                                      'maximum pixel width in the '+var+' direction. If this is '+
                                      'intended, no further action is necessary. Otherwise, adjust '+
                                      'the [wrap_around_thresh].')
                bnds_tmp[-1,1] = bnds_tmp[0,0]
        # Add to ds
        ds[var+'_bnds'] = bnds_tmp
        del bnds_tmp

lat/lon bounds not found in dataset; they will be created.
HERE IS THE ISSUE PROBABLY
HERE IS THE ISSUE PROBABLY


In [14]:
bnds_tmp = xr.DataArray(data=np.zeros((ds.dims[var],2))*np.nan,
                                dims=[var,'bnds'],
                                coords=[ds[var],np.arange(0,2)])
bnds_tmp

<xarray.DataArray (lon: 532, bnds: 2)>
array([[nan, nan],
       [nan, nan],
       [nan, nan],
       ...,
       [nan, nan],
       [nan, nan],
       [nan, nan]])
Coordinates:
  * lon      (lon) float64 -115.0 -114.9 -114.8 -114.7 ... -62.1 -62.0 -61.9
  * bnds     (bnds) int64 0 1

In [24]:
bnds_tmp

<xarray.DataArray (lon: 532, bnds: 2)>
array([[      nan,       nan],
       [-114.9511, -114.8511],
       [-114.8511, -114.7511],
       ...,
       [ -62.1511,  -62.0511],
       [ -62.0511,  -61.9511],
       [ -61.9511,  -61.8511]])
Coordinates:
  * lon      (lon) float64 -115.0 -114.9 -114.8 -114.7 ... -62.1 -62.0 -61.9
  * bnds     (bnds) int64 0 1

In [25]:
bnds_tmp[1:,:] = xr.concat([ds[var]-0.5*ds[var].diff(var),
                                      ds[var]+0.5*ds[var].diff(var)],dim='bnds').transpose(var,'bnds')
a = bnds_tmp.copy()

bnds_tmp[1:,:] = xr.concat([ds[var]-0.5*ds[var].diff(var),
                                      ds[var]+0.5*ds[var].diff(var)],dim='bnd').transpose(var,'bnd')

b = bnds_tmp.copy()

xr.testing.assert_allclose(a,b)

In [22]:
xr.concat([ds[var]-0.5*ds[var].diff(var),
                                      ds[var]+0.5*ds[var].diff(var)],dim='bnds').transpose(var,'bnds')

<xarray.DataArray 'lon' (lon: 531, bnds: 2)>
array([[-114.9511, -114.8511],
       [-114.8511, -114.7511],
       [-114.7511, -114.6511],
       ...,
       [ -62.1511,  -62.0511],
       [ -62.0511,  -61.9511],
       [ -61.9511,  -61.8511]])
Coordinates:
  * lon      (lon) float64 -114.9 -114.8 -114.7 -114.6 ... -62.1 -62.0 -61.9
Dimensions without coordinates: bnds